In [1]:
import pandas as pd
import numpy as np

## Merging

Let's exploring joining DataFrames in Python with the Pandas `merge()` method.

For this exercise, let's go to the Pandas package documentation to learn about how to use the method and follow along.

https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=merge#pandas.merge

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

The dataset is provided to you in several csv files that each represent a table form a relational database.
    * Students: dataframe with one record per student
    * Classes: dataset with one record per class
    * Enrollment: dataset with one record per student class pair
    * Grades: grade for each class

### Read Data

In [2]:
students = pd.read_csv('students.csv')
classes = pd.read_csv('classes.csv')
enrollment = pd.read_csv('enrollment1.csv')
grades = pd.read_csv('grades.csv')

## Pair Programming

For each of the following, before writing code - write out:
    * what are the common keys we can use to join the tables?
    * what type of join should we use?

##### Questions:

1. How many classes is each student taking? Join the `students` and `enrollment` tables

2. How many units is each student taking? Join the `students`, `enrollment` and `classes` tables

3. How classes are shared between each student and professor pair?

4. Whats the average number of students each professor has in class?

### What if the keys don't match?


In [3]:
student_enroll = pd.merge(students, enrollment, 
                    left_on='student_id', right_on='student_id')

In [4]:
students.head()

,id,name,favorite_color,age,graduation_year,student_id
0,0,Kingsley,green,83,2022,0
1,1,Garrick,green,79,2022,1
2,2,Tom,green,40,2022,2
3,3,Arthur,pink,52,2022,3
4,4,Frank,red,95,2021,4


In [5]:
student_enroll = pd.merge(students, enrollment, 
                    left_on='student_id', right_on='student_id')

student_enroll_classes = pd.merge(student_enroll, classes, 
                                left_on='class_id', right_on='class_id')

In [6]:
student_enroll_classes.head()

,id_x,name,favorite_color,age,graduation_year,student_id,id_y,class_id,id,units,professor
0,0,Kingsley,green,83,2022,0,14,INFO-2003,3,2,Rubeus
1,0,Kingsley,green,83,2022,0,46,INFO-2003,3,2,Rubeus
2,0,Kingsley,green,83,2022,0,102,INFO-2003,3,2,Rubeus
3,0,Kingsley,green,83,2022,0,107,INFO-2003,3,2,Rubeus
4,1,Garrick,green,79,2022,1,132,INFO-2003,3,2,Rubeus


In [7]:
student_enroll_classes.groupby('name')['units'].sum()

name
Amos        16
Arthur      20
Frank       18
Garrick     26
Gregory     24
Kingsley    40
Rowena      18
Terry       24
Tom         22
vincent     20
Name: units, dtype: int64

## Classes Shared between student-prof pair

In [8]:
x = student_enroll_classes.groupby(['name','professor'])['class_id'].count()

x.unstack()

professor,Dolores,Horace,Quirinus,Rubeus
name,,,,
Amos,3.0,NaN,1.0,1.0
Arthur,2.0,1.0,2.0,2.0
Frank,1.0,1.0,1.0,3.0
Garrick,2.0,4.0,NaN,1.0
Gregory,3.0,1.0,1.0,3.0
Kingsley,1.0,4.0,3.0,4.0
Rowena,2.0,3.0,NaN,NaN
Terry,1.0,2.0,2.0,2.0
Tom,3.0,3.0,NaN,1.0


## Extra (if we have time)

Let's also explore the Pandas Concatenate `concat()` method

The documentation for the method is here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html?highlight=concat#pandas.concat

We found an error when we pulled data earlier, and we only took some of the enrollment records. When we fixed our query, we found another file of additional records. How can we join them to our original enrollment?

1. Join Enrollment Records using concatenate method

2. If we wanted to know the average grade per class - can we use the concatenate method to join `grades` and `enrollment`? ) Why or why not?

In [9]:
enrollment_part1 = pd.read_csv('enrollment1.csv')
enrollment_part2 = pd.read_csv('enrollment2.csv')

In [10]:
# Q1
enrollment = pd.concat([enrollment_part1, enrollment_part2], axis=0)

In [11]:
grades

,id,class_id,student_id,grade
0,101,INFO-2003,0,A
1,28,INFO-2000,0,A
2,44,INFO-2000,0,B
3,46,INFO-2003,0,B
4,59,INFO-2000,0,C
...,...,...,...,...
145,21,INFO-2001,9,B
146,145,INFO-2000,9,C
147,147,INFO-2002,9,A
148,128,INFO-2000,9,C


In [12]:
# Q2
grades = pd.read_csv('grades.csv')

In [13]:
# Whats the problem if we concatenate these?
#merged_grade_enroll = pd.concat([grades, enrollment], axis=1)

In [14]:
# We would need to reset the index first (assuming this was okay)
enrollment = enrollment.reset_index()
merged_grade_enroll = pd.concat([grades, enrollment], axis=1)

In [15]:
# We could sort and join... but this is still problematic - why?
grades = grades.sort_values(by=['class_id','student_id'])
enrollment = enrollment.sort_values(by=['class_id','student_id'])

In [16]:
# Better: join on multiple keys
merged = pd.merge(grades, enrollment, on = ['class_id', 'student_id'], suffixes=('_grades', '_enrollment'))

In [17]:
merged.head()

,id_grades,class_id,student_id,grade,index,id_enrollment
0,28,INFO-2000,0,A,12,28
1,28,INFO-2000,0,A,18,44
2,28,INFO-2000,0,A,28,62
3,28,INFO-2000,0,A,54,108
4,28,INFO-2000,0,A,33,59
